In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [32]:
from keras_lr_multiplier import LRMultiplier

In [33]:
from keras.models import load_model

import matplotlib.pyplot as plt
from keras.layers import Conv2D, Dropout, Dense, Flatten, BatchNormalization, MaxPooling2D, Activation
from keras.optimizers import Adam, Adadelta, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

import keras.backend as K

In [34]:
Dig_MNIST = pd.read_csv("Dig-MNIST.csv")
sample_submission = pd.read_csv("sample_submission.csv")
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

**Pre-trained model** on original MNIST https://github.com/kkweon/mnist-competition/tree/master/model

In [35]:
model = load_model('vggnet5.h5')

In [36]:
batch_size =1024  # 64
num_classes = 10
epochs = 10 # 50
learning_rate=1e-2

In [37]:
model.summary()

Model: "VGG5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization_11 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
activation_11 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 28, 28, 32)        9248      
_________________________________________________________________
batch_normalization_12 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
activation_12 (Activation)   (None, 28, 28, 32)        0      

In [38]:
mult_dict={}

In [39]:
dict_len=0

In [40]:
for layer in model.layers:
    if layer.trainable and layer.weights:
        dict_len+=1
        

In [41]:
dict_len

23

In [42]:
lr=learning_rate

In [43]:
muls =[lr/(dict_len - i) for i in range(dict_len)]

In [44]:
muls = iter(muls)

In [45]:
muls

In [46]:
for layer in model.layers:
    if layer.trainable and layer.weights:
        mult_dict[layer.name]=next(muls)
        

In [47]:
mult_dict

{'block1_conv1': 0.0004347826086956522,
 'batch_normalization_11': 0.00045454545454545455,
 'block1_conv2': 0.0004761904761904762,
 'batch_normalization_12': 0.0005,
 'block2_conv1': 0.0005263157894736842,
 'batch_normalization_13': 0.0005555555555555556,
 'block2_conv2': 0.0005882352941176471,
 'batch_normalization_14': 0.000625,
 'block3_conv1': 0.0006666666666666666,
 'batch_normalization_15': 0.0007142857142857143,
 'block3_conv2': 0.0007692307692307692,
 'batch_normalization_16': 0.0008333333333333334,
 'block3_conv3': 0.0009090909090909091,
 'batch_normalization_17': 0.001,
 'block4_conv1': 0.0011111111111111111,
 'batch_normalization_18': 0.00125,
 'block4_conv2': 0.0014285714285714286,
 'batch_normalization_19': 0.0016666666666666668,
 'block4_conv3': 0.002,
 'batch_normalization_20': 0.0025,
 'fc-1': 0.0033333333333333335,
 'fc-2': 0.005,
 'predictions': 0.01}

In [48]:
# Split the data
x = train.iloc[:,1:].values
y = train.iloc[:,0].values
y[:10]
x_test = test.drop('id', axis=1).iloc[:,:].values
x_dig = Dig_MNIST.drop('label', axis=1).iloc[:,:].values
y_dig = Dig_MNIST.label

# Reshape the data
x = x.reshape(x.shape[0], 28, 28, 1)
y = to_categorical(y, 10)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_dig = x_dig.reshape(x_dig.shape[0], 28, 28, 1)

# Split the data between train and test
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.10, random_state=1) 

# Artificially increase training set
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=10,
                                   width_shift_range=0.25,
                                   height_shift_range=0.25,
                                   shear_range=0.1,
                                   zoom_range=0.25,
                                   horizontal_flip=False)

valid_datagen = ImageDataGenerator(rescale=1./255.,
                                  rotation_range=10,
                                   width_shift_range=0.25,
                                   height_shift_range=0.25,
                                   shear_range=0.1,
                                   zoom_range=0.25,
                                   horizontal_flip=False)

I have tried with lesser Learning Rates, what I observed was increasing the lr imporved accuracy with less epochs. So I have used lr=1e-2

In [49]:
# def get_lr_metric(optimizer):
#     def lr(y_true, y_pred):
#         return optimizer.lr
#     return lr

In [50]:
optimizer=LRMultiplier('adam', mult_dict)

In [51]:
# lr_metric = get_lr_metric(optimizer)
# model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['acc', lr_metric])

In [52]:
# reducing the learning rate by half every 2 epochs
# cbks = [LearningRateScheduler(lambda epoch: 0.001 * 0.5 ** (epoch // 2)),
#         TensorBoard(write_graph=False)]
# X = np.random.rand(1000, 50)
# Y = np.random.randint(2, size=1000)
# model.fit(X, Y, epochs=10, callbacks=cbks)

In [53]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy']) #lr_metric

In [54]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=200,
                                            verbose=1,
                                            factor=0.2)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)

In [55]:
%%time
history = model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size, shuffle=True),
                              steps_per_epoch=100,
                              epochs=epochs,
                              validation_data=valid_datagen.flow(x_valid, y_valid),
                              validation_steps=50,
                              callbacks=[learning_rate_reduction, es])

{'block1_conv1': 0.0004347826086956522, 'batch_normalization_11': 0.00045454545454545455, 'block1_conv2': 0.0004761904761904762, 'batch_normalization_12': 0.0005, 'block2_conv1': 0.0005263157894736842, 'batch_normalization_13': 0.0005555555555555556, 'block2_conv2': 0.0005882352941176471, 'batch_normalization_14': 0.000625, 'block3_conv1': 0.0006666666666666666, 'batch_normalization_15': 0.0007142857142857143, 'block3_conv2': 0.0007692307692307692, 'batch_normalization_16': 0.0008333333333333334, 'block3_conv3': 0.0009090909090909091, 'batch_normalization_17': 0.001, 'block4_conv1': 0.0011111111111111111, 'batch_normalization_18': 0.00125, 'block4_conv2': 0.0014285714285714286, 'batch_normalization_19': 0.0016666666666666668, 'block4_conv3': 0.002, 'batch_normalization_20': 0.0025, 'fc-1': 0.0033333333333333335, 'fc-2': 0.005, 'predictions': 0.01} 0 [<tf.Variable 'Adam/iterations:0' shape=() dtype=int64, numpy=0>, <tf.Variable 'm_0:0' shape=(3, 3, 1, 32) dtype=float32, numpy=
array([[[

InvalidArgumentError: cannot compute Equal as input #1(zero-based) was expected to be a float tensor but is a int64 tensor [Op:Equal]

In [56]:
model.save('TmodelVgg5.h5')

In [57]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [ ]:
predictions = model.predict(x_test/255.)

In [ ]:
preds = np.argmax(predictions, axis = 1)

In [ ]:
submission = pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')
submission['label'] = preds
submission.to_csv("submission.csv", index=False)